In [14]:
from langchain_google_genai import GoogleGenerativeAI
import google.generativeai as genai
import os
from dotenv import find_dotenv,load_dotenv
from langchain import HuggingFaceHub

load_dotenv(find_dotenv())

HUGGINGFACEHUB_API_TOKEN=os.getenv("HUGGINGFACEHUB_API_TOKEN")
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

llm=GoogleGenerativeAI(model="gemini-pro", temperature=0.3,google_api_key=GOOGLE_API_KEY)
llm_huggingface=HuggingFaceHub(repo_id="google/flan-t5-base",model_kwargs={"temperature":0.3,"max_length":64})

output=llm.invoke("Where is the capital of China")
print(output)

Beijing


### Chain

In [9]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
user_template="where is the captial of {nation}"
prompt=PromptTemplate(template=user_template,input_variables=['nation'])
chain = LLMChain(llm=llm_huggingface, prompt=prompt)
chain.run(nation="china")


'shanghai'

### Sequential Chain

In [10]:
from langchain.chains import SimpleSequentialChain

chain1=LLMChain(llm=llm_huggingface,prompt=PromptTemplate(template="where is the capital of {nation}",input_variables=['nation']))
chain2=LLMChain(llm=llm_huggingface,prompt=PromptTemplate(template="suggest me some places to visit in {capital}",input_variables=['capital']))
chain=SimpleSequentialChain(chains=[chain1,chain2])
chain.run('china')

'qingdao'

### Chat model

In [11]:
from langchain.schema import HumanMessage,SystemMessage,AIMessage
from langchain_google_genai import ChatGoogleGenerativeAI
chatllm = ChatGoogleGenerativeAI(model="gemini-pro",temperature=0.3,google_api_key=GOOGLE_API_KEY,convert_system_message_to_human=True)
chatllm([
SystemMessage(content="Yor are a comedian AI assitant"),
HumanMessage(content="Please provide some comedy punchlines on AI")
])

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


AIMessage(content="1. Why did the AI cross the road? To get to the other algorithm.\n\n2. What do you call an AI that's always telling jokes? A pun-isher.\n\n3. Why did the AI get a degree in computer science? To become a byte-lingual.\n\n4. What's the difference between an AI and a human? An AI doesn't need a coffee break to process information.\n\n5. Why did the AI get a job as a programmer? Because it was tired of being a know-it-all.\n\n6. What do you call an AI that's always making mistakes? A bug-droid.\n\n7. Why did the AI get a job as a therapist? To help people with their algorithm-related problems.\n\n8. What's the difference between an AI and a politician? An AI doesn't make promises it can't keep.\n\n9. Why did the AI get a job as a teacher? To help students learn about the future.\n\n10. What do you call an AI that's always getting into trouble? A byte-mare.")

### Prompt Template + LLM + Output Parsers

In [12]:
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

class Commaseperatedoutput(BaseOutputParser):
    def parse(self,text:str):
        return text.strip().split(",")
    
template="Your are a helpful assistant. When the use given any input , you should generate 5 words synonyms in a comma seperated list"
human_template="{text}"
chatprompt=ChatPromptTemplate.from_messages([
    ("system",template),
    ("human",human_template)
])

chain=chatprompt|chatllm|Commaseperatedoutput()
chain.invoke({"text":"intelligent"})


['smart', ' clever', ' bright', ' astute', ' sharp']